# Running an application

Now that we have developed our machine learning model in notebooks, we want to deploy it as a service. The purpose is to allow data science exploration to easily transition into deployed services and applications on the OpenShift platform.  After saving this project to git, it can be built on the OpenShift platform to serve models.


There are a lot of files in this folder. We will only need to edit the noteboks, but here's insight into what each of these files do: 

### Project Organization
```
.
├── README.md
├── LICENSE
├── requirements.txt        <- Used to install packages for s2i application
├── 0_start_here.ipynb      <- Instructional notebook
├── 1_run_flask.ipynb       <- Notebook for running flask locally to test
├── 2_test_flask.ipynb      <- Notebook for testing flask requests
├── .gitignore              <- standard python gitignore
├── .s2i                    <- hidden folder for advanced s2i configuration
│   └── environment         <- s2i environment settings
├── gunicorn_config.py      <- configuration for gunicorn when run in OpenShift
├── prediction.py           <- the predict function called from Flask
└── wsgi.py                 <- basic Flask application
```

We will use a source to image build, also known as s2i, to deploy the model service, but first let's practice running the service in a notebook: 

## Install Dependencies

In [1]:
import sys
!{sys.executable} -m pip install -r requirements.txt

     |████████████████████████████████| 95 kB 5.1 MB/s             
     |████████████████████████████████| 79 kB 26.6 MB/s            
     |████████████████████████████████| 20.6 MB 34.6 MB/s            1��██████▏            | 12.4 MB 34.6 MB/s eta 0:00:01|██████████████████████████      | 16.7 MB 34.6 MB/s eta 0:00:01
     |████████████████████████████████| 26.0 MB 70.4 MB/s            �██████████▋               | 13.5 MB 70.4 MB/s eta 0:00:01███████████████████████▊     | 21.7 MB 70.4 MB/s eta 0:00:01
     |████████████████████████████████| 10.0 MB 39.7 MB/s            █▌                             | 768 kB 39.7 MB/s eta 0:00:01�████████████▎        | 7.2 MB 39.7 MB/s eta 0:00:01
     |████████████████████████████████| 7.0 MB 45.7 MB/s             1.1 MB 45.7 MB/s eta 0:00:016.3 MB 45.7 MB/s eta 0:00:01
     |████████████████████████████████| 63.2 MB 61.9 MB/s            MB 61.9 MB/s eta 0:00:01019 MB/s eta 0:00:01�██████████▉               | 33.1 MB 61.9 MB/s eta 0:00:01MB/s eta 

## Experiment

Experiment with data and create your prediction function.  Create any serialized models needed.

In [2]:
import cloudpickle as cp
import pandas as pd

pipeline = cp.load(open('pipeline.pkl', 'rb'))

In [3]:
def predict(args_dict):

    d = {'timestamp':0, 'label':0, 'user_id': args_dict.get('user_id'), 'amount': args_dict.get('amount'), 'merchant_id': args_dict.get('merchant_id'), 'trans_type': args_dict.get('trans_type'), 'foreign': args_dict.get('foreign'), 'interarrival': args_dict.get('interarrival')}
    
    df = pd.DataFrame(d, index=[0])
    prediction = pipeline.predict(df)[0]

    return {'prediction': prediction}

In [4]:
my_req = {"user_id": 1698, "amount": 7915, "merchant_id": 22.37, "trans_type": "contactless", "foreign": "False", "interarrival": 9609}

predict(my_req)

{'prediction': 'legitimate'}

## Create a Predict Function

Now that we are certain our prediction function is working, we can go ahead and add it to the prediction.py file. We've already done this for you, but go take a look and make sure you see what's happening. 

Also, make sure `requirements.txt` is updated with any additional packages you've used and need for prediction.

## Test Predict Function

In [5]:
from prediction import predict


my_req = {"user_id": 1698, "amount": 7915, "merchant_id": 22.37, "trans_type": "contactless", "foreign": "False", "interarrival": 9609}

predict(my_req)



{'prediction': 'legitimate'}

## Try to get a prediction of fraud

Now, try it again, but this time, change the values until the prediction returns **fraud** instead of **legitimate**. 

In [6]:
other_req = {"user_id": 9999, 
             "amount": 9999, 
             "merchant_id": 99999, 
             "trans_type": "contactless", 
             "foreign": "False", 
             "interarrival": 1}

predict(other_req)

{'prediction': 'legitimate'}